In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LCEL for chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

model = ChatOpenAI(
    model="gpt-4o-mini"
)

# StrOutputParser() ai 의 응답을 내용만 봅아서 볼 수 있음. 원래는 메타 데이터도 포함됨.
chain = prompt | model | StrOutputParser()

print(f"[답변]: {chain.invoke({'topic': 'ice cream'})}")



In [ ]:
# chain 의 stream 기능 써보기
model = ChatOpenAI(
    model="gpt-4o-mini"
)

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

chain = prompt | model

for s in chain.stream({"topic": "ice cream"}):
    print(s.content, end="", flush=True)


Stream 사용 이유
* 빠른 응답 제공: 전체 응답이 생성될 때까지 기다릴 필요 없이, 한 토큰씩 받아볼 수 있음.
* 실시간 처리 가능: UI에서 점진적으로 텍스트를 표시하거나, 스트림 데이터를 다른 곳에서 활용할 수 있음.
* 메모리 효율성: 한 번에 모든 데이터를 로드하지 않기 때문에 메모리 부담이 줄어듦.